A

In [5]:
import sys
input = lambda: sys.stdin.readline().rstrip()

inf = float('inf')

def get_tm(H, M, h, m):
    t1 = H * 60 + M
    t2 = h * 60 + m
    
    if t1 > t2:
        t2 += 24 * 60
        
    diff = t2 - t1
    return diff

for _ in range(int(input())):
    n, H, M = map(int, input().split())
    li = [list(map(int, input().split())) for _ in range(n)]

    min_tm = inf
    for h, m in li:
        min_tm = min(min_tm, get_tm(H, M, h, m))

    print(min_tm // 60, min_tm % 60)

2 23 35
20 15
10 30
10 55


B

In [9]:
import sys
input = lambda: sys.stdin.readline().rstrip()

for _ in range(int(input())):
    n = int(input())
    li = list(map(int, input().split()))

    st = set()
    result = 0
    for i in range(n - 1, -1, -1):
        if li[i] in st:
            result = i + 1
            break
        st.add(li[i])

    print(result)

5
1 1 1 1 1
4


C

In [ ]:
9 8 7 6 5 4 3 2 1

In [13]:
from itertools import combinations as comb
import sys
input = lambda: sys.stdin.readline().rstrip()

inf = float('inf')

min_cases = [inf] * 46
for num in range(1, 10):
    for case in comb(range(1, 10), num):
        sv = sum(case)
        case = sorted(case)
        ncase = int(''.join(map(str, case)))
        min_cases[sv] = min(min_cases[sv], ncase)
        
for _ in range(int(input())):
    s = int(input())
    
    r = min_cases[s]
    
    print(r)

4
20
389
8
8
45
123456789
10
19


D

In [19]:
import sys
input = lambda: sys.stdin.readline().rstrip()

for _ in range(int(input())):
    s = input()
    s_len = len(s)

    n = int(input())
    li = [input() for _ in range(n)]

    to_index = dict()
    for i in range(n):
        to_index[li[i]] = i + 1
    st = set(li)

    to_start = 0
    to_end = 0

    total = 0
    seq = []

    ip = True
    while True:
        max_end = -1
        max_end_start = -1
        max_end_seq = ''
        for start in range(to_start, to_end + 1):
            for end in range(start, start + 10):
                if end == s_len:
                    break
                if end <= max_end:
                    continue
                temp = s[start:end + 1]
                if temp in st:
                    max_end_start = start
                    max_end_seq = temp
                    max_end = end

        if max_end == -1:
            ip = False
            break

        total += 1
        seq.append((to_index[max_end_seq], max_end_start + 1))
        to_start = to_end + 1
        to_end = max_end + 1

        if to_end == s_len:
            break

    if not ip:
        print(-1)
    else:
        print(total)
        for v in seq:
            print(*v)

bababa
2
ba
aba
3
1 1
2 2
2 4


E

In [32]:
import sys
# input = lambda: sys.stdin.readline().rstrip()

for _ in range(int(input())):
    n = int(input())
    li = list(map(int, input().split()))

    st = {0, 8}
    for i in range(n):
        while True:
            rem = li[i] % 10
            if rem not in st:
                li[i] += rem
            else:
                break

    ip = True
    is8 = False
    is0 = False
    zeros = set()
    for v in li:
        rem = v % 10

        if rem == 0:
            is0 = True
            zeros.add(v)
        else:
            is8 = True

    if is0 and is8:
        ip = False
    elif is0:
        if len(zeros) == 1:
            ip = True
        else:
            ip = False
    else:
        nli = {v % 20 for v in li}

        if len(nli) == 1:
            ip = True
        else:
            ip = False

    if ip:
        print('Yes')
    else:
        print('No')

1
2
50 30
Yes


G

In [75]:
from heapq import heappush, heappop
import sys
input = lambda: sys.stdin.readline().rstrip()

## PYRIVAL BOOTSTRAP
# https://github.com/cheran-senthil/PyRival/blob/master/pyrival/misc/bootstrap.py
# This decorator allows for recursion without actually doing recursion
## @bootstrap, yield when getting and returning value in recursive functions, end of functions
from types import GeneratorType

def bootstrap(f, stack=[]):
    def wrappedfunc(*args, **kwargs):
        if stack:
            return f(*args, **kwargs)
        else:
            to = f(*args, **kwargs)
            while True:
                if type(to) is GeneratorType:
                    stack.append(to)
                    to = next(to)
                else:
                    stack.pop()
                    if not stack:
                        break
                    to = stack[-1].send(to)
            return to
    return wrappedfunc

@bootstrap
def find(v):
    if v != parent[v]:
        parent[v] = yield find(parent[v])
    yield parent[v]

def union(a, b):
    if len(heaps[a]) < len(heaps[b]):
        a, b = b, a
        
    parent[b] = a
    while heaps[b]:
        heappush(heaps[a], heappop(heaps[b]))
    
@bootstrap
def dfs(value, depth, ta, tb):
    depths[value] = depth
    
    for v, a, b in li[value]:
        yield dfs(v, depth + 1, ta + a, tb + b)
        rvalue = yield find(value)
        rv = yield find(v)
                 
        union(rvalue, rv)
    
    rvalue = yield find(value)
    
    heappush(heaps[rvalue], (-ta, value))
    
#     print(value, heaps[rvalue], tb)
    
    while heaps[rvalue]:
        if tb <= -heaps[rvalue][0][0]:
            check_ta, vv = heappop(heaps[rvalue])
#             print(value, vv, -check_ta, tb)

            result[vv] = depth
        else:
            break
            
#     print(value, heaps[rvalue])

    
    yield
    
for _ in range(int(input())):
    n = int(input())
    li = [[] for _ in range(n)]
    for child in range(1, n):
        p, a, b = map(int, input().split())
        p -= 1

        li[p].append((child, a, b))

    depths = [-1] * n
    heaps = [[] for _ in range(n)]
    parent = list(range(n))
    result = [0] * n

    dfs(0, 0, 0, 0)

    print(*result[1:])

0 3 1 2 1 1 2 3
